# Problem Formulation

## Problem Description:
This project aims to develop a machine learning model that predicts future sales
and demand by utilising historical sales data and external factors, including
product details, promotions, seasonality, holidays, and economic indicators. The
goal is to analyse historical patterns and generate reliable forecasts that help
businesses make data-driven decisions to reduce costs, increase efficiency, and
improve customer satisfaction by predicting the daily sales for the next 28 days.

## Objectives

● Collect and preprocess historical sales and demand data.

● Identify key features that influence sales trends.

● Build, train, and optimise forecasting models to predict future sales and
demand.

● Deploy the best-performing model to generate forecasts in real-time or in
batches.
## Data source:
Hierarchical sales data from Walmart, the world’s largest company by revenue in the US.



# Code setup

## Important libraries

In [1]:
import pandas as pd
import numpy as np

## data reading

In [ ]:
sales_validation=pd.read_csv('/kaggle/input/m5-forecasting-accuracy/sales_train_validation.csv')
sales_validation.head(3)

In [ ]:
cal=pd.read_csv('/kaggle/input/m5-forecasting-accuracy/calendar.csv')
cal.head(3)

In [ ]:
sell_price = pd.read_csv("/kaggle/input/m5-forecasting-accuracy/sell_prices.csv")
sell_price.head(3)

In [2]:
temp_data = pd.read_csv("/kaggle/input/depi-dataset/temp_data.csv")

# EDA

In [ ]:
cal.info()

In [4]:
temp_data.head(3)

,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap
0,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,No event,No event,No event,No event,0
1,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,No event,No event,No event,No event,0
2,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,No event,No event,No event,No event,0


In [ ]:
cal.head(200)

# Schema formating

## sales_validation file

In [ ]:
data = sales_validation.melt(
    id_vars=["id","item_id", "dept_id","cat_id","store_id","state_id"],  # columns to keep
    var_name="d",                          # new column name for day labels (d_1, d_2, ...)
    value_name="sales"                     # new column name for sales values
)



In [ ]:
data.drop(columns=['id'],inplace=True)

## calendar file

In [ ]:
cal["date"]= pd.to_datetime(cal["date"])

In [ ]:
cal["event_name_1"]= cal["event_name_1"].fillna("No event")
cal["event_type_1"]= cal["event_type_1"].fillna("No event")
cal["event_name_2"]= cal["event_name_2"].fillna("No event")
cal["event_type_2"]= cal["event_type_2"].fillna("No event")

In [ ]:
# Merge data and cal dataframes on the 'd' column
merged_data = pd.merge(data, cal, on='d', how='left')

conditions = [
    merged_data["state_id"] == "CA",
    merged_data["state_id"] == "TX",
    merged_data["state_id"] == "WI"
]
choices= [
    merged_data["snap_CA"],
    merged_data["snap_TX"],
    merged_data["snap_WI"]
]
merged_data["snap"]= np.select(conditions, choices)

merged_data.drop(columns=['snap_CA','snap_TX','snap_WI'],inplace=True)